In [1]:
import keras 
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras import backend as k

#TRAINING AND TESTING THE DATA
(x_train,y_train),(x_test,y_test)=mnist.load_data()

x_train=x_train.reshape(x_train.shape[0],28,28,1)
x_test=x_test.reshape(x_test.shape[0],28,28,1)
input_shape=(28,28,1)

#CONVERTING CLASS VECTORS INTO bINARY
y_train=keras.utils.to_categorical(y_train,10)
y_test=keras.utils.to_categorical(y_test,10)

x_train=x_train.astype("float32")
x_test=x_test.astype("float32")

x_train/=255
x_test/=255

batch_size=128
num_classes=10

epochs=2

model=Sequential()
model.add(Conv2D(kernel_size=5,filters=1,activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(kernel_size=5,filters=1,activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))

#COMPLIE THE MODEL
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])
hist=model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))

score=model.evaluate(x_test,y_test,verbose=0)

print("loss",score[0])
print("accuacy",score[1])

model.save('mnist.h5')


import keras
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab,Image
import numpy as np

model=load_model('mnist.h5')
def predict_digit(img):
    #resize image to 28x28 pixels

    img=img.resize((28,28))

    #convert rgb to grayscale

    img=img.convert('L')
    img=np.array(img)

    #reshaping to support our model input and normalizing

    img=img.reshape(1,28,28,1)
    img=img/255.0

    #predicting the class
    res =model.predict([img])[0]
    returnp.argmax(res),max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)                 
        
        self.x=self.y=0

       #creating Elements
        self.canvas=tk.Canvas(self,width=300,height=300,bg="white",cursor="cross")
        self.label=tk.Label(self,text="Draw..",font=("Helvetica",48))
        self.classify_btn=tk.Button(self,text="Recognise",command=self.classify_handwriting)
        self.button_clear=tk.Button(self,text="clear",command=self.clear_all)


       #GRID STRUCTURE

        self.canvas.grid(row=0,column=0,pady=2, sticky=w,)
        self.label.grid(row=0,column=1,pady=2,padx=2)
        self.classify_btn.grid(row=1, column=1,pady=2,padx=2)
        self.button.clear.grid(row=1,column=0,pady=2)
        self.canvas.bind("<B1-Motion>",self.draw_lines)


def clear_all(self):
    self.canvas.delete("all")

def classify_handwriting(self):
    HWND=self.canvas.winfo_id()  #get the handle of the canvas
    rect=win32gui.GetWindowReact(HWND)  #get the coordinate of the canvas
    a,b,c,d=rect
    rect=(a+4,b+4,c-4,d-4)   
    im = ImageGrab.grab(rect) 
    digit,acc = predict_digit(im)
    self.label.configure(text=str(digit)+','+str(int(acc*100))+'%')


def draw_lines(self,event):
    self.x = event.x
    self.y=event.y

    r=8
    self.canvas.create_oval(self.x-r,self.x+r,self.y+r,fill='')


    app=App()
    mainloop()

Epoch 1/2
469/469 [==============================] - 6s 11ms/step - loss: 2.3082 - accuracy: 0.1055 - val_loss: 2.3016 - val_accuracy: 0.1216
Epoch 2/2
469/469 [==============================] - 5s 11ms/step - loss: 2.3064 - accuracy: 0.1064 - val_loss: 2.3005 - val_accuracy: 0.1256
loss 2.3004934787750244
accuacy 0.12559999525547028
